In [ ]:
import os
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
  sys.path.append(module_path)

%env OPENAI_API_KEY = "*"
from PIL import Image
from IPython.core.display import HTML
from functools import partial

from engine.utils import ProgramGenerator, ProgramInterpreter
from prompts.gqa_agent import *
from engine.step_interpreters import parse_step
import openai

In [ ]:
prompter = partial(create_prompt_stepname_selective)
generator = ProgramGenerator(prompter=prompter)
interpreter = ProgramInterpreter(dataset='gqa')

In [ ]:
def run_case(image_path, question):
    image = Image.open(image_path)
    image.thumbnail((640,640),Image.Resampling.LANCZOS)
    state = dict(
        IMAGE=image.convert('RGB')
    )
    caption, _, _ = interpreter.execute('X=CAP(image=IMAGE)',state,inspect=True)
    state['context'] = f'multimodal input: IMAGE ({image.size[0]}wx{image.size[1]}h)\n\nCAPTION_IMAGE="{caption}"\n\nQ="{question}"\n\n<End of Context>\n\nThought(Analyze the question: '
    shots = 0
    success = False
    direct_VQA_result, prog_state, html_str = interpreter.execute(f'X=VQA(image=IMAGE,question="{question}")',state,inspect=True)
    display(HTML(html_str))
    while shots < 15:
        response, _ = generator.generate(dict(input=state['context']))
        state['context'] += response + '))'
        try:
            statement = response.split('Implement(')[1] + ')'
            result, prog_state, html_str = interpreter.execute(statement,state,inspect=True)
            display(HTML(html_str))
            step_name = parse_step(statement)['step_name']
            if 'RESULT' in step_name:
              return result, state['context']
            elif 'CROP' not in step_name:
              state['context'] += '\n<result>\n' + str(result) + '\n</result>\nThought(Analyze the result: '
            else:
              output_var = parse_step(statement)['output_var']
              caption, _, _ = interpreter.execute(f'X=CAP(image={output_var})',state,inspect=True)
              state['context'] += '\n<result>\n' + caption + '\n</result>\nThought(Analyze the result: '
        except:
          response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                  {
                    "role": "user",
                    "content": f"Read the following problem analysis procedure. It is incomplete, and you need to deduce the most likely answer. If I ask the VQA system directly about the IMAGE, it will respond with \"{direct_VQA_result}.\" Based on all the available information, **PROVIDE YOUR BEST GUESS**. Note that the accuracy of VQA is not 100% so you should have an analysis based on both the following procedure and the VQA result.\n{state['context']}"
                  }
                ]
              )
          response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                  {
                    "role": "user",
                    "content": f"Read the following analysis, DIRECTLY give me the answer to the question WITHOUT any analysis.\n{response['choices'][0]['message']['content']}"
                  }
                ]
              )
          return response['choices'][0]['message']['content'], state['context']
        shots += 1
    response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                  {
                    "role": "user",
                    "content": f"Read the following problem analysis procedure. It is incomplete, and you need to deduce the most likely answer. If I ask the VQA system directly about the IMAGE, it will respond with \"{direct_VQA_result}.\" Based on all the available information, **PROVIDE YOUR BEST GUESS**. Note that the accuracy of VQA is not 100% so you should have an analysis based on both the following procedure and the VQA result.\n{state['context']}"
                  }
                ]
              )
    response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
            {
              "role": "user",
              "content": f"Read the following analysis, DIRECTLY give me the answer to the question WITHOUT any analysis.\n{response['choices'][0]['message']['content']}"
            }
          ]
        )
    return response['choices'][0]['message']['content'], state['context']